## Suggested Solver

**Note: This is only a suggested way of solving the problem. This solution is in no way complete and needs significant improvements, something we hope the competition participants will be able to do.**

**Step 1.** First convert all features into objective features - this makes modeling easier. To do this, assign a numeric value to every string. For example ```'F13': {'DB1': 4, 'DB2': 2, 'DB3': 0, 'DB4': 3, 'DB5': 1}```

**Step 2.** Calculate benchmark returns for every period as wi`*`Ri and break end of period TRR into three buckets: over perform index(TRR>wi`*`Ri), under perform index, +ve(TRR<wi`*`Ri, TRR>0), underperform index -ve (TRR<wi`*`Ri, TRR<0)
Create a target variable `Y` and assign the following values: over perform index: 1 under perform index but +ve return: 0 underperform index and -ve return: -1

**Step 3.** Train a model which uses the given features to predict which bucket the performance will be in, use F to predict `Y`. Train a few models to check which one is the best.

**Step 4.** Define a new optimization objective function as wt`*`Ypred - transaction costs (We want as much weight as possible to be assigned to highest bucket). Run a model that first uses all the features (obviously not TRR!) to generate Ypred and then an optimizer to maximize this reward while meeting constraints.

Below is a simple implementation of this model. Suggested improvements could be:
1. Break performance into more granular buckets
2. Use a better training mechanism to predict Y (or more feature engineering before predictions
3. Define a better optimization objective function
4. Use a better optimization method


In [1]:
import pandas as pd
import numpy as np
import os.path
try:
    from urllib2 import urlopen
except ImportError:
    from urllib.request import urlopen

In [2]:
## This calculates the reward, Do not change this function
def getReward(wt, wt_1, ri, l, k):
    if (ri is None) or (wt is None):
        port_returns.append(0)
        sharpe_ratio.append(0)
        port_volatility.append(0)
        reward = 0
    else:
        ri.fillna(0, inplace=True)
        returns = np.dot(wt, ri)
        port_returns.append(returns)
        downside_returns = [x for x in port_returns if x < 0]
        volatility = np.std(downside_returns) #np.sqrt(np.dot(weights.T, np.dot(cov_annual, weights)))
        sharpe = np.sqrt(12)*np.mean(port_returns) / volatility if volatility!=0 else 0
        sharpe_ratio.append(sharpe)
        port_volatility.append(volatility)
        if (wt_1 is None):
            phi = 0
        else:
            ix = wt.index | wt_1.index
            tn = wt.reindex(ix) - wt_1.reindex(ix)
            tn[tn.isnull()&tn.index.isin(wt.index)] = wt
            tn[tn.isnull()&tn.index.isin(wt_1.index)] = -wt_1
            phi = k*tn.abs().sum()
        reward = returns - l*volatility - phi
#         print('returns, volatility, phi, sharpe, reward')
#         print(returns, volatility, phi, sharpe, reward)
    return reward

## Do not change this function, this verifies if all constraints are met
def checkConstraints(wt, wt_1, wi, Dt, St, Qt, g, U, t, T, P, delta, chi, eta):
    violated = False
    tol = 0.009
    if np.abs(wt.sum()-1)>tol:
        print(wt.sum())
        print("Fully Invested Constraint Violated: Sum of weights is not 1")
        violated = True
    div_constraint = np.maximum(g, 1/float(Qt.sum()))
    if (wt.abs()-div_constraint>tol).any():
        print(wt[(wt.abs()-div_constraint>tol)])
        print("Diversification Constraint Violated: All weights are not less than parameter %.2f"%div_constraint )
        violated = True
    if wt_1 is None:
        turnover = 0
    else:
        ix = wt.index | wt_1.index
        tn = wt.reindex(ix) - wt_1.reindex(ix)
        tn[tn.isnull()&tn.index.isin(wt.index)] = wt
        tn[tn.isnull()&tn.index.isin(wt_1.index)] = -wt_1
        turnover = (tn).abs().sum()/2
    turnover_list.append(turnover)
    if (np.sum(turnover_list[-12:])>U):
        print("%0.2f Turnover Constraint Violated: Turnover Limit exceeded"%np.sum(turnover_list[-12:]))
        violated = True
    if (wt<t).any():
        print("Shortsell Constraint Violated: all weights are not greater than parameter t")
        violated = True
    if wt[wt<0].sum()<T:
        print("Max Shortsell Constraint Violated: sum of all weights are not greater than parameter T")
        violated = True
    if wt[wt!=0].count() < np.minimum(P, len(wt)):
        print("Min number of positions Constraint Violated: count of all weights <>0 %i are not greater than parameter P%i"%(wt[wt!=0].count(), np.minimum(P, len(wt))))
        violated = True
    if (wt*Dt).sum()/ (wi*Dt).sum() > delta:
        print("Duration Constraint Violated: wt*Dt/ wi*Dt %.2f is greater than parameter delta"%((wt*Dt).sum()/ (wi*Dt).sum()))
        violated = True
    if (wt*St).sum()/ (wi*St).sum() > chi:
        print("Spread Constraint Violated: wt*St/ wi*St %.2f is greater than parameter chi"%((wt*St).sum()/ (wi*St).sum()))
        violated = True
    if (wt*(1-Qt)).abs().sum()>tol:
        print("Qualification Constraint Violated: wt*(1-qt) is not zero %.2f"%(wt*(1-Qt)).abs().sum())
        violated = True
#     if returns - Rlow/ volatility <= np.sqrt(1-eta):
#         print("Max Risk probability Constraint Violated: returns - Rlow/ volatility <= np.sqrt(1-eta)")
            
    return violated

## Do not change this function, this verifies if final constraints are met
def checkFinalConstraints(Rmin, volmax):
    violated = False
    if np.sum(port_returns)<Rmin:
        print("Total Return Constraint Violated: Total Return is less than Index Return")
        violated = True
    if port_volatility[-1]>volmax:
        print("Volatility Constraint Violated: Vol is higher than Index Vol")
        violated = True
    if sharpe_ratio[-1]<Rmin/volmax:
        print("Sharpe Ratio Constraint Violated: SR is less than Index SR")
        violated = True
        return violated

In [3]:
def getSymbolsToTrade():
    ################################################
    ####   COPY FROM BELOW INTO TEMPLATE FILE   ####
    ################################################
    
    return 'G1'

In [4]:
index = getSymbolsToTrade()
if not os.path.isfile('%s.csv'%index):
    downloadFile(index)
idx_data = pd.read_csv('%s.csv'%index, index_col='TimeStamp')
idx_data.sort_index(axis=0, level=None, ascending=True, inplace=True)
print(idx_data.size)
idx_data=idx_data[~idx_data.duplicated()]
print(idx_data.size)
print("Data Column Names:")
print(idx_data.columns)
print(index, idx_data.index[0], idx_data.index[-1])
print(idx_data.head(10))

3416422
3411322
Data Column Names:
Index([u'AssetGroup', u'Identifier', u'F2', u'F3', u'F4', u'F5', u'q', u'F7',
       u'F8', u'd', u'S', u'F11', u'F12', u'F13', u'F14', u'F15', u'F16',
       u'F17', u'wI', u'F19', u'TRR', u'F21', u'F22', u'F23', u'F24', u'F25',
       u'F26', u'F27', u'F28', u'F29', u'F30', u'F31', u'F32', u'F33'],
      dtype='object')
('G1', '1998-12-31', '2018-11-30')
           AssetGroup Identifier        F2                  F3  \
TimeStamp                                                        
1998-12-31         G1   86a1f1ee  bc068363       Capital Goods   
1998-12-31         G1   de5b9bca  e5fb34b9  Telecommunications   
1998-12-31         G1   2a0a4ee3  62835655      Basic Industry   
1998-12-31         G1   a67fb965  81f811c5      Transportation   
1998-12-31         G1   52f697cc  2becce58      Basic Industry   
1998-12-31         G1   bf46c4bb  57847ea5  Telecommunications   
1998-12-31         G1   e39dffda  9c69fc3a  Telecommunications   
1998-12-31  

## Suggested Solver

Step 1. convert all features into objective features

In [5]:
for c in idx_data.columns:
    if idx_data[c].dtype==object:
        print(c, idx_data[c].dtype)

('AssetGroup', dtype('O'))
('Identifier', dtype('O'))
('F2', dtype('O'))
('F3', dtype('O'))
('F4', dtype('O'))
('F7', dtype('O'))
('F12', dtype('O'))
('F13', dtype('O'))


In [6]:
listf = ['F3', 'F4', 'F7', 'F12', 'F13']
feature_dict ={}
for feature in listf:    
    fs = idx_data[feature].unique()
    f_dict = {}
    count = 0
    for f in fs:
        idx_data[feature][idx_data[feature]==f] = count
        f_dict[f] = count
        count = count+1
    feature_dict[feature] = f_dict

/Users/chandinijain/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
feature_dict

{'F12': {'1500-2500': 0, '2500-3500': 3, '3500+': 1, '500-1500': 2, '<500': 4},
 'F13': {'DB1': 4, 'DB2': 2, 'DB3': 0, 'DB4': 3, 'DB5': 1},
 'F3': {'Agency': 14,
  'Automotive': 8,
  'Basic Industry': 2,
  'Capital Goods': 0,
  'Consumer Goods': 6,
  'Energy': 5,
  'Government Guaranteed': 12,
  'Healthcare': 17,
  'Leisure': 16,
  'Local-Authority': 7,
  'Media': 4,
  'Real Estate': 11,
  'Retail': 10,
  'Services': 13,
  'Technology & Electronics': 15,
  'Telecommunications': 1,
  'Transportation': 3,
  'Utility': 9},
 'F4': {'Advertising': 39,
  'Aerospace/Defense': 42,
  'Agency': 26,
  'Air Transportation': 48,
  'Auto Loans': 46,
  'Auto Parts & Equipment': 32,
  'Automakers': 14,
  'Beverage': 17,
  'Building & Construction': 6,
  'Building Materials': 4,
  'Cable & Satellite TV': 5,
  'Chemicals': 2,
  'Department Stores': 44,
  'Diversified Capital Goods': 0,
  'Electric-Distr/Trans': 25,
  'Electric-Generation': 21,
  'Electric-Integrated': 15,
  'Electronics': 31,
  'Energy 

#### Step 2. break end period TRR into three buckets: over perform index, under perform index, +ve, underperform index -ve
Create a target variable 'Y' and assign the following values:
over perform index: 1
under perform index but +ve return: 0
underperform index and -ve return: -1

In [9]:
dates = idx_data.index.unique()
idx_returns = pd.Series(0.0,index=dates)
counter=0
new_idx_data = pd.DataFrame(columns=idx_data.columns)
while counter < len(dates):
    date = dates[counter]
    date_data = idx_data[idx_data.index == date]
    
    ## get all the identifiers for a date
    
    date_data.set_index( date_data['Identifier'], inplace=True)
    date_data = date_data[~date_data.index.duplicated()]
    
    wi = pd.Series(date_data['wI']/100, index = date_data['Identifier'])
    ri = pd.Series(date_data['TRR'], index = date_data['Identifier'])
    idx_returns[date]=np.dot(wi, ri)
    
    date_data.loc[date_data['TRR']<idx_returns[date], 'Y'] = 0
    date_data.loc[date_data['TRR']<0, 'Y'] = -1
    date_data.loc[date_data['TRR']>=idx_returns[date], 'Y'] = 1
    new_idx_data = new_idx_data.append(date_data)
    counter = counter+1


#### Step 3. Train a model which uses the given features to predict which bucket the performance will be in. Train a few models to check which one is the best

In [10]:
X = new_idx_data.copy()
del X['TRR']
del X['AssetGroup']
del X['Identifier']
del X['Y']
del X['F2']
y = new_idx_data['Y'].copy()
X['NAs'] = X.isnull().sum(axis=1)
X = X.replace([np.inf, -np.inf], np.nan)
X.fillna(-50, inplace=True)


In [11]:
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split
# dividing X, y into train and test data 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 10) 

In [12]:
 # training a DescisionTreeClassifier 
from sklearn.tree import DecisionTreeClassifier 
dtree_model = DecisionTreeClassifier(max_depth = 3).fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test) 

# accuracy on X_test 
accuracy = dtree_model.score(X_test, y_test) 
print accuracy

# creating a confusion matrix 
cm = confusion_matrix(y_test, dtree_predictions)
cm

0.6314004070718762


array([[ 4025,     0,  2237],
       [  383,     0,  5377],
       [ 1239,     0, 11796]])

In [13]:
# # dividing X, y into train and test data 
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0) 
  
# # training a linear SVM classifier 
# from sklearn.svm import SVC 
# svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train) 
# svm_predictions = svm_model_linear.predict(X_test) 
  
# # model accuracy for X_test   
# accuracy = svm_model_linear.score(X_test, y_test) 
  
# # creating a confusion matrix 
# cm = confusion_matrix(y_test, svm_predictions) 
# cm

In [14]:
# dividing X, y into train and test data 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0) 
  
# training a KNN classifier 
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier(n_neighbors = 7).fit(X_train, y_train) 
  
# accuracy on X_test 
accuracy = knn.score(X_test, y_test) 
print accuracy 
  
# creating a confusion matrix 
knn_predictions = knn.predict(X_test)  
cm = confusion_matrix(y_test, knn_predictions) 
cm

0.4657780261004909


array([[1541,  852, 3831],
       [ 867, 1662, 3271],
       [2405, 2160, 8468]])

In [15]:
# dividing X, y into train and test data 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0) 
  
# training a Naive Bayes classifier 
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB().fit(X_train, y_train) 
gnb_predictions = gnb.predict(X_test) 
  
# accuracy on X_test 
accuracy = gnb.score(X_test, y_test) 
print accuracy 
  
# creating a confusion matrix 
cm = confusion_matrix(y_test, gnb_predictions) 
cm

0.39074909207008024


array([[ 634, 3120, 2470],
       [  76, 5060,  664],
       [ 504, 8432, 4097]])

In [16]:
### Taking ensemble of all three

y_avg = (gnb_predictions+knn_predictions+dtree_predictions)/3
y_avg = [1 if x>0.5 else x for x in y_avg]
y_avg = [-1 if x<-0.5 else x for x in y_avg]
y_avg = [0 if (x>=-0.5)and(x<=0.5) else x for x in y_avg]
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_avg))
cm = confusion_matrix(y_test, y_avg) 
cm


0.393861994652193


array([[ 197, 3201, 2826],
       [ 161, 3124, 2515],
       [ 351, 6134, 6548]])

#### Step 4. Define a new optimization objective function as wt*bucketValue - transaction costs (We want as much weight as possible to be assigned to highest bucket). Train an optimizer to maximize this reward while meeting constraints

In [17]:
#     ### In this sample case, I am using scipy.optimize 
from scipy.optimize import minimize
# import cvxpy as cp

In [18]:
## Here I define my objective function 

class CustomFeatures():
    
    def newFeature1(self, wt, wt_1, ri, k):
    ################################################
    ####   COPY FROM BELOW INTO TEMPLATE FILE   ####
    ################################################
#         ri = kwargs['ri']
#         wt = kwargs['wt']
#         wt_1 = kwargs['wt_1']
#         l = kwargs['l']
        ri.fillna(0, inplace=True)
        returns = np.dot(wt, ri)
#         volatility = np.std(port_returns) #np.sqrt(np.dot(weights.T, np.dot(cov_annual, weights)))
        weights = pd.Series(wt, index = ri.index)
    
        if (wt_1 is None):
            phi = 0
        else:
            ix = weights.index | wt_1.index
            tn = weights.reindex(ix) - wt_1.reindex(ix)
            tn[tn.isnull()&tn.index.isin(weights.index)] = weights
            tn[tn.isnull()&tn.index.isin(wt_1.index)] = -wt_1
            phi = k*tn.abs().sum()
        reward = -(returns - phi)
        return reward

In [48]:
def getWeights(identifiers, reward, wi, Dt, St, Qt, g, U, t, T, P, delta, chi, eta, df, trr, wt_1,**kwargs):
    ################################################
    ####   COPY FROM BELOW INTO TEMPLATE FILE   ####
    ################################################
    
    ### I predict future returns as returns from t-1
    ri = getPrediction(identifiers, wi, Dt, St, Qt, g, U, t, T, P, delta, chi, eta, df=df)
    
    div_constraint = np.maximum(g, 1/float(Qt.sum()))
    
    allowed = Qt[Qt==1].index   ###pre=process to assign weights only where Qt==1, this takes care of last constraint automatically
    
    bounds = ((t,div_constraint),) * len(allowed) #limits on weights
    
    cons = ({'type': 'eq', 'fun': lambda wt: 1.0 - np.sum(wt)},   ##sum of weights should be 1
            {'type': 'ineq', 'args': (P, Qt,),'fun': lambda wt, P, Qt: np.count_nonzero(wt) - np.minimum(P, Qt.sum())}, ## minimum number of positions should be P
            {'type': 'ineq', 'args': (delta, wi, Dt,),'fun': lambda wt, delta, wi, Dt: delta - (wt*Dt[Qt==1]).sum()/ (wi*Dt[Qt==1]).sum()}, # duration should be within a % of index
            {'type': 'ineq', 'args': (chi, wi, St,),'fun': lambda wt, chi, wi, St: chi - (wt*St[Qt==1]).sum()/ (wi*St[Qt==1]).sum()}, # spread should be within a % of index
#             {'type': 'eq', 'args': (Qt,),'fun': lambda wt, Qt: (wt*(1-Qt[Qt==1])).sum()} #only trade Q==1 assets
           )
    ## optimization function is defined in custom feature
    CustomFeaturesCls = CustomFeatures()
    weights = minimize(CustomFeaturesCls.newFeature1, wi[Qt==1], args=(wt_1, ri[Qt==1], k,), method='SLSQP', bounds=bounds,constraints=cons,
                      options={'maxiter': 500, 'ftol': 1e-06, 'iprint': 1, 'disp': True})   
    print(weights.x)
    
    w = pd.Series(0, index = date_data['Identifier'])
    w[qt==1] = pd.Series(weights.x , index = allowed) ## assets with q==1 are assigned weights, rest are zero
    print(w)
    return w

In [50]:
def getKwargs():   
    return {}

In [51]:
## Step 3: Optional: Fill in the logic to return predictions for return on asset
## This function takes in the same inputs as getWeights()

### do not change the inputs to the function. If you want any extra inputs, specify them as **kwargs
### you can lookup this tutorial on how to use **kwargs https://www.geeksforgeeks.org/args-kwargs-python/

def getPrediction(identifiers, wi, Dt, St, Qt, g, U, t, T, P, delta, chi, eta, **kwargs):
    ################################################
    ####   COPY FROM BELOW INTO TEMPLATE FILE   ####
    ################################################
    
    X = kwargs['df'].copy()
    del X['AssetGroup']
    del X['Identifier']
    del X['F2']
    X['NAs'] = X.isnull().sum(axis=1)
    X = X.replace([np.inf, -np.inf], np.nan)
    X.fillna(-50, inplace=True)
    y1 = dtree_model.predict(X)
    y2 = knn.predict(X)
    y3 = gnb.predict(X)
    
    y_avg = (y1+y2+y3)/3
    y_avg = [1 if x>0.5 else x for x in y_avg]
    y_avg = [-1 if x<-0.5 else x for x in y_avg]
    y_avg = [0 if (x>=-0.5)and(x<=0.5) else x for x in y_avg]
    
    ri = pd.Series(y_avg, index = date_data['Identifier'])
    
    return ri

In [54]:
# empty lists to store returns, volatility and weights of imaginary portfolios
port_returns = []
port_volatility = []
sharpe_ratio = []
asset_weights = []
reward_list = []
turnover_list = []
idx_returns = []

#port_returns ,port_volatility ,sharpe_ratio ,asset_weights ,reward_list ,turnover_list ,idx_returns 

#empty df to store previous period returns
ri = None

#empty dict to store values by date
dict_metrics_by_date = {}


In [ ]:
### Evaluator to getweights at everytime t and calcuate reward + check if constraints are met

## specifying all the constants
counter = 0
l = 0.1
k = 0.03
g = 0.04
U = 2.5
t= 0
T = 0
P = 50
delta = 0.5
chi = 0.15
eta = 0.95

## initializing arrays
dates = idx_data.index.unique()
wi = None

## looping over all dates
while counter < len(dates):
    date = dates[counter]
    
    print('################# \nDATE %s'%date)
    
    ## load all the data for a date
    date_data = idx_data[idx_data.index == date]
    
    ## get all the identifiers for a date
    
    date_data.set_index( date_data['Identifier'], inplace=True)
    date_data = date_data[~date_data.index.duplicated()]
    cusips = date_data['Identifier']
    
    ## old weights
    wt_1 = asset_weights[-1] if len(asset_weights)>0 else None
    wt_2 = asset_weights[-2] if len(asset_weights)>1 else None
    
    ##old index weights
    wi_t_1 = None if wi is None else wi.copy()
    
    ## calculate reward at start of time t from weights allocated at time t-1
    reward = getReward(wt_1, wt_2, ri, l, k)
    
    ## calculate index return at start of time t from weights allocated at time t-1
    if ri is None or wi_t_1 is None:
        idx_returns.append(0)
    else:
        idx_returns.append(np.dot(wi_t_1, ri))
    
    #load specific feature info for time t
    wi = pd.Series(date_data['wI']/100, index = date_data['Identifier'])
    Dt = pd.Series(date_data['d'], index = date_data['Identifier'])
    St = pd.Series(date_data['S'], index = date_data['Identifier'])
    qt = pd.Series(date_data['q'], index = date_data['Identifier'])
    
    ## get new weights
    wt = getWeights(cusips, reward, wi, Dt, St, qt, g, U, t, T, P, delta, chi, eta, \
                        date_data.drop(columns=['TRR']), ri, wt_1, **getKwargs()) 

    
    ## store relevant info in their lists
    asset_weights.append(wt)
    reward_list.append(reward)
    
    ## verify if all constraints are met
    if checkConstraints(wt, wt_1, wi, Dt, St, qt, g, U, t, T, P, delta, chi, eta):
        print("ERROR!!!! weights don't meet contraints, exiting")
#         break
    
    dict_metrics_by_date[date] = {'returns': port_returns[-1],
                                  'volatility': port_volatility[-1] ,
                                  'Sharpe Ratio': sharpe_ratio[-1],
                                  'Index Returns': idx_returns[-1],
                                  'Reward': reward_list[-1],
                                  '12m turnover' : turnover_list[-1],
                                  'weights' :asset_weights[-1]}
    
    print(dict_metrics_by_date[date])
    
    counter += 1
    
    ## Store end of month returns to calculate reward in next period
    del ri
    ri = pd.Series(date_data['TRR'], index = date_data['Identifier'])

## Calculate returns for last period    
if ri is not None:
    reward = getReward(wt, wt_1, ri, l, k)
    reward_list.append(reward)
    idx_returns.append(np.dot(wi, ri))
## check if contraints on total return and risk are met        
Rmin = np.sum(idx_returns)
volmax = np.std(idx_returns)
if checkFinalConstraints(Rmin, volmax):
    print("ERROR!!!! weights don't meet return/risk limit contraints, exiting")
else:
    print("Portfolio Metrics:")
    print("Total Return: %.2f"%np.sum(port_returns))
    print("Standard Deviation: %.2f"%port_volatility[-1])
    print("Sharpe Ratio: %.2f"%sharpe_ratio[-1])
    

################# 
DATE 1998-12-31
Iteration limit exceeded    (Exit mode 9)
            Current function value: -0.0
            Iterations: 501
            Function evaluations: 48226
            Gradient evaluations: 498
[5.05763090e-14 4.00000000e-02 1.01864238e-16 8.59771324e-13
 1.29978003e-12 2.56686996e-16 6.81948952e-12 1.29060012e-16
 1.25836280e-14 4.00000000e-02 1.27139991e-16 7.61719321e-12
 4.00000000e-02 1.14353317e-12 6.79731849e-17 3.44398312e-16
 1.78002800e-12 4.00000000e-02 1.96922421e-12 4.00000000e-02
 3.06247717e-12 6.85838326e-14 5.66210716e-17 4.00000000e-02
 1.32169238e-13 9.50336953e-13 4.14557983e-13 4.00000000e-02
 4.00000000e-02 1.65976519e-16 4.00000000e-02 1.04615822e-16
 4.00000000e-02 1.72246821e-12 8.28112578e-17 2.31095487e-13
 3.41866126e-02 1.47401416e-12 4.00000000e-02 5.23737820e-12
 2.65268150e-12 2.43377491e-12 1.01823018e-16 6.68527746e-17
 8.60391186e-17 1.63301017e-12 9.26332282e-17 1.42029188e-12
 1.86783362e-16 1.33664804e-16 3.01451123e-1

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.0120477953324
            Iterations: 76
            Function evaluations: 7138
            Gradient evaluations: 72
[4.00000000e-02 8.81215444e-16 1.02076253e-14 9.97992631e-15
 2.10856389e-12 4.00000000e-02 2.15598198e-12 8.89780286e-15
 1.52606905e-12 4.08347655e-12 3.07934071e-17 4.72620280e-13
 4.63746673e-12 1.57832607e-14 4.00000000e-02 1.47674307e-12
 1.89380371e-12 1.38128998e-12 1.44259119e-15 4.30295394e-12
 1.95499202e-15 5.26599937e-12 4.00000000e-02 7.53455267e-15
 1.38517302e-12 3.27956657e-02 4.00000000e-02 2.87745401e-12
 7.83769103e-15 1.08945950e-11 9.46012882e-12 4.00000000e-02
 4.85871096e-12 7.66813156e-13 4.22518389e-12 4.00000000e-02
 9.66641255e-15 7.33862630e-13 3.22053555e-13 1.44092663e-15
 4.00000000e-02 2.27943560e-15 4.00000000e-02 3.84596913e-15
 4.00000000e-02 1.34649563e-12 1.34550607e-12 1.82233637e-12
 4.00000000e-02 4.00000000e-02 9.03501784e-12 3.31869771e-

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.0147251031289
            Iterations: 102
            Function evaluations: 9866
            Gradient evaluations: 98
[2.53456155e-12 3.64597843e-12 1.93717053e-17 4.00000000e-02
 5.75647963e-18 1.10529861e-11 6.23317763e-12 3.51425287e-12
 1.07522880e-17 4.00000000e-02 4.00000000e-02 1.15385548e-12
 6.21905056e-12 1.14964239e-17 2.73829227e-18 4.00000000e-02
 2.46249199e-17 4.97341442e-18 6.76775893e-18 3.42931623e-02
 4.80124791e-12 6.92647811e-18 1.66108909e-17 6.83358180e-18
 4.00000000e-02 1.42863742e-17 4.21713612e-12 4.00000000e-02
 2.37689992e-18 3.25146494e-18 4.98979523e-13 2.04116111e-12
 4.83274754e-18 7.60684379e-13 4.00000000e-02 7.05505630e-18
 4.68758514e-12 1.23287201e-12 2.47360193e-18 2.97926412e-17
 5.06426769e-12 2.32225048e-18 3.07619502e-13 1.31188172e-17
 2.57173826e-12 7.51329313e-18 1.42886966e-17 8.39879117e-12
 4.00000000e-02 1.34532594e-12 1.21384142e-12 7.21583716e

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.00662182245534
            Iterations: 236
            Function evaluations: 24070
            Gradient evaluations: 232
[1.83960221e-16 4.00000000e-02 8.55309401e-12 1.39320910e-15
 1.28584747e-16 8.50050644e-13 9.37175867e-17 3.51860990e-12
 4.00000000e-02 8.96573286e-17 4.00000000e-02 1.20849582e-16
 5.03004908e-16 7.94849409e-17 5.85544237e-17 4.00000000e-02
 9.66456671e-13 3.64169776e-17 4.00000000e-02 6.36538384e-14
 4.00000000e-02 1.75659676e-15 4.00000000e-02 7.15369987e-17
 7.51537268e-03 4.00000000e-02 2.93738654e-17 1.52161747e-12
 4.26371607e-17 1.68935643e-12 5.39459925e-12 2.75039397e-12
 4.08905026e-16 4.00000000e-02 2.29240411e-12 1.79780250e-12
 4.21831707e-13 9.24504825e-17 3.92983421e-16 8.94555360e-17
 4.00000000e-02 1.93529513e-12 7.80229270e-13 3.58410863e-16
 1.43068530e-11 1.54608135e-13 1.95112573e-15 5.81441908e-14
 4.00000000e-02 2.44593730e-16 9.39961860e-17 4.000000

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.00720108658586
            Iterations: 49
            Function evaluations: 4595
            Gradient evaluations: 45
[9.99670181e-12 2.71257819e-02 4.27745456e-12 4.49776912e-17
 3.95647671e-13 1.12462495e-11 4.00000000e-02 6.61936926e-17
 1.15623293e-15 7.14943221e-13 8.22007856e-12 2.74481629e-12
 4.00000000e-02 5.33730207e-12 6.09780931e-03 4.41236836e-18
 2.28310552e-16 8.15233674e-18 3.25910890e-17 1.45140407e-15
 1.07396597e-11 5.21289881e-17 4.00000000e-02 2.44887086e-12
 2.24064586e-12 2.59366955e-16 4.00000000e-02 4.00000000e-02
 3.38127743e-12 4.66706495e-12 1.09042533e-11 9.26390998e-18
 7.26715794e-16 3.33515657e-17 1.23162794e-11 3.26584905e-12
 6.77384752e-12 2.15584540e-16 4.00000000e-02 4.00000000e-02
 8.75203950e-12 6.51746711e-16 1.38417285e-17 4.00000000e-02
 4.00000000e-02 1.18488548e-17 1.87406407e-12 5.22336819e-17
 1.01943926e-11 4.01119931e-16 1.12770537e-17 4.00000000e

...

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.0322822008811
            Iterations: 57
            Function evaluations: 17086
            Gradient evaluations: 53
[9.93245122e-18 2.06431355e-17 6.14103733e-12 1.26887850e-17
 1.64694649e-16 5.64314898e-12 2.55535183e-17 2.92897486e-17
 1.23123107e-11 5.13880240e-12 1.69118523e-12 8.70921632e-17
 8.57015077e-13 3.85773826e-17 9.50469170e-17 4.60491305e-12
 1.08869208e-16 7.33558388e-12 4.96472025e-17 1.39652101e-12
 2.13782835e-17 1.36495728e-17 5.85417966e-17 4.00000000e-02
 6.31116459e-17 3.10231435e-11 4.40138599e-17 4.32423798e-12
 8.94268682e-12 5.04233960e-18 8.11706311e-12 3.78438187e-12
 7.96056311e-18 7.87851404e-18 1.32919503e-11 1.70259793e-17
 1.41921040e-17 9.77382425e-18 4.11212627e-18 4.47145492e-17
 2.53037092e-18 5.09151313e-17 1.33713266e-17 1.20989686e-17
 2.92080651e-17 4.00000000e-02 3.39505478e-13 9.07717878e-14
 2.54108108e-12 2.88629331e-12 4.01388398e-12 5.87272784

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.0369077001552
            Iterations: 86
            Function evaluations: 27286
            Gradient evaluations: 82
[1.04996907e-17 3.70127250e-16 2.18377785e-11 2.09704659e-11
 1.55239135e-14 6.24427523e-12 3.47755670e-12 2.30381545e-12
 6.14865532e-12 8.22787835e-12 2.48516953e-16 7.15216609e-17
 1.68104416e-13 1.13456388e-16 1.45594021e-16 1.38494735e-12
 1.34192250e-17 2.96037291e-02 1.12713054e-16 3.28207466e-16
 2.03193002e-11 2.67441599e-11 4.19317861e-11 5.34310990e-13
 4.00000000e-02 1.37030927e-17 4.53767939e-12 3.98884842e-17
 1.47829551e-16 4.15612905e-12 1.61874342e-16 3.37981898e-13
 1.32379421e-17 1.13939565e-17 1.26391802e-16 5.75691903e-17
 1.08621729e-11 3.73519086e-12 1.50239215e-16 3.18922928e-12
 1.80736835e-11 5.47509302e-13 1.01885295e-11 5.99102631e-17
 7.81641918e-12 1.58199886e-11 4.00000000e-02 7.11743264e-12
 1.47096091e-11 6.93181964e-12 1.23020109e-16 6.46672995

Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: -0.147299653751
            Iterations: 63
            Function evaluations: 19939
            Gradient evaluations: 59
[4.85748032e-11 2.27456475e-15 2.57483021e-11 4.85628757e-15
 4.66729880e-15 3.45502004e-10 1.23878636e-11 1.84626052e-11
 2.61508536e-11 2.35293237e-15 2.82446214e-15 7.09121489e-12
 1.13771373e-13 6.64630296e-14 1.25794653e-13 5.69600148e-14
 1.57366589e-11 9.99552820e-11 5.19846336e-11 2.12073082e-11
 8.41891586e-15 4.00000000e-02 2.61943936e-14 1.10731681e-11
 7.57301435e-15 1.74118835e-14 9.51863560e-15 1.05010181e-13
 1.94140093e-14 4.00000000e-02 5.10971120e-11 3.15658991e-11
 4.81861488e-11 1.76516207e-14 9.69365849e-12 2.23978868e-14
 1.03895528e-13 8.97743664e-15 8.55532933e-11 3.38272456e-12
 5.18618299e-14 1.04910746e-14 4.00000000e-02 1.22674531e-14
 6.30512477e-11 2.60301760e-14 1.89254378e-14 3.61639094e-11
 2.56246179e-11 1.06266460e-14 8.75667864e-15 3.

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.0731296659864
            Iterations: 75
            Function evaluations: 25540
            Gradient evaluations: 71
[1.07524507e-12 1.38675708e-11 2.28971453e-12 3.26690903e-15
 4.46617029e-17 8.34648822e-12 1.19661348e-12 2.71812016e-18
 1.19524944e-11 2.25946194e-13 3.47411961e-18 6.94307817e-18
 2.24653169e-17 4.20107008e-16 8.66985668e-18 2.56187778e-11
 1.29178140e-15 7.92661239e-18 1.55645292e-13 1.62499019e-11
 1.05338627e-11 1.66046858e-12 4.46878392e-18 7.57077339e-13
 9.63677654e-16 2.63695626e-12 1.59331834e-17 4.00000000e-02
 1.18233777e-16 4.39705490e-12 5.23214676e-18 7.21158990e-12
 1.18393557e-11 3.83221114e-18 5.28600881e-12 4.07183664e-16
 9.17746559e-17 6.16525737e-18 4.00000000e-02 9.94012471e-12
 3.74848979e-18 5.59068172e-16 5.33276864e-18 4.00000000e-02
 4.00000000e-02 4.00000000e-02 1.06255888e-16 2.55870425e-13
 3.95147268e-18 4.55560415e-12 2.49092988e-16 3.73876534

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.0138033591416
            Iterations: 108
            Function evaluations: 39289
            Gradient evaluations: 104
[1.88099279e-12 3.36084095e-14 1.66394006e-11 1.83562633e-16
 9.59886065e-13 1.29442756e-16 1.73691648e-12 4.62307603e-12
 5.75223807e-17 5.11744271e-12 4.51583670e-12 1.20148739e-14
 4.00000000e-02 7.65835672e-13 4.63690929e-16 1.68353143e-12
 1.69256278e-16 9.51734497e-14 1.70898746e-15 1.08546947e-16
 2.23517342e-17 3.78149715e-12 4.80288722e-15 4.34679473e-12
 5.53837910e-16 1.78406819e-11 2.94082724e-15 8.63001255e-12
 2.50758191e-11 1.62660029e-16 1.93466677e-11 1.18294618e-11
 8.63349910e-12 4.36765002e-16 4.89211047e-15 4.20525816e-16
 9.81963993e-14 4.11497575e-17 9.08540406e-16 1.00490099e-15
 6.70013610e-16 1.08953765e-11 6.10937577e-15 7.60572573e-16
 1.74561419e-16 2.22839799e-11 2.41476867e-17 1.47982653e-11
 3.99438754e-16 5.84713253e-14 1.30230777e-11 5.1263404